<a href="https://colab.research.google.com/github/dawikrol/Engineering-thesis/blob/main/Scripts/Wind_potential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Wind potential**





### Lib import

In [1]:
import pandas as pd
from google.colab import files
import io
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Uploading data

You find data in project's GitHub repository in Data/Wind.

Data regarding atmospheric conditions come from: https://ec.europa.eu/jrc/en/pvgis



In [2]:
uploaded = files.upload()

Saving Limanowski_01_W.csv to Limanowski_01_W.csv
Saving Nowosądecki_01_W.csv to Nowosądecki_01_W.csv
Saving Nowotarski_01_W.csv to Nowotarski_01_W.csv
Saving NowySącz_01_W.csv to NowySącz_01_W.csv
Saving Tatrzański_01_W.csv to Tatrzański_01_W.csv


### Creating raw dataframes

In [15]:
dataframes = {}

for file in uploaded:
    name, region, kind  = file.split('_')
    dataframes[name] = pd.read_csv(io.BytesIO(uploaded[file]), error_bad_lines=False,
                                   index_col=None, header=None, sep='\n')

In [16]:
dataframes[list(dataframes.keys())[0]].size

8787

In [18]:
dataframes[list(dataframes.keys())[0]].head(30)

,0
0,Latitude (decimal degrees): 49.723
1,Longitude (decimal degrees): 20.395
2,Elevation (m): 400.0
3,"month,year"
4,"1,2006"
5,"2,2010"
6,"3,2006"
7,"4,2012"
8,"5,2012"
9,"6,2011"


In [19]:
dataframes[list(dataframes.keys())[0]].tail(30)

,0
8757,"20101231:0400,-7.55,69.97,0.0,-0.0,0.0,220.02,..."
8758,"20101231:0500,-7.22,70.29,0.0,-0.0,0.0,223.4,3..."
8759,"20101231:0600,-6.39,70.01,0.0,-0.0,0.0,233.68,..."
8760,"20101231:0700,-5.57,69.74,3.0,0.0,3.0,243.97,3..."
8761,"20101231:0800,-4.75,69.46,67.0,23.3,63.0,254.2..."
8762,"20101231:0900,-4.11,70.33,109.0,28.05,102.0,26..."
8763,"20101231:1000,-3.46,71.2,140.0,44.83,127.0,266..."
8764,"20101231:1100,-2.82,72.07,136.0,34.45,126.0,27..."
8765,"20101231:1200,-3.06,76.09,147.0,119.81,117.0,2..."
8766,"20101231:1300,-3.31,80.12,74.0,0.0,74.0,275.47..."


### Creating list of shortcuts

In [36]:
info_df = dataframes[list(dataframes.keys())[0]].iloc[8777:8786]
info_df.columns = ['defu']
info_df = info_df.defu.str.split(':', expand=True)
info_df.columns = ['symbol', 'meaning']
info_df.set_index('symbol', inplace=True)
info_df


,meaning
symbol,
T2m,2-m air temperature (degree Celsius)
RH,relative humidity (%)
G(h),Global irradiance on the horizontal plane (W/m2)
Gb(n),Beam/direct irradiance on a plane always norm...
Gd(h),Diffuse irradiance on the horizontal plane (W...
IR(h),Surface infrared (thermal) irradiance on a ho...
WS10m,10-m total wind speed (m/s)
WD10m,"10-m wind direction (0 = N, 90 = E) (degree)"
SP,Surface (air) pressure (Pa)


### Data cleaning

In [22]:
cleared_df = {}

for key in dataframes:
    cl_data = dataframes[key].iloc[16:8777]
    cl_data.columns = ['defu'] #the default column name is inconvenient
    cl_data = cl_data.defu.str.split(',', expand=True)

cl_data

,0,1,2,3,4,5,6,7,8,9
16,time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP
17,20110101:0000,-8.39,0.5,0.0,-0.0,0.0,239.95,3.77,252.0,90407.0
18,20110101:0100,-8.56,0.5,0.0,-0.0,0.0,240.68,3.8,247.0,90389.0
19,20110101:0200,-8.73,0.5,0.0,-0.0,0.0,241.41,3.83,241.0,90371.0
20,20110101:0300,-8.89,0.5,0.0,-0.0,0.0,242.14,3.85,241.0,90324.0
...,...,...,...,...,...,...,...,...,...,...
8772,20121231:1900,-7.55,99.0,0.0,-0.0,0.0,236.3,3.63,251.0,90846.0
8773,20121231:2000,-7.72,0.5,0.0,-0.0,0.0,237.03,3.66,255.0,90858.0
8774,20121231:2100,-7.88,0.5,0.0,-0.0,0.0,237.76,3.68,259.0,90867.0
8775,20121231:2200,-8.05,0.5,0.0,-0.0,0.0,238.49,3.71,264.0,90876.0
